# 🧊 Cold Atom Fringe Remover – Research Tutorial

The goal of this tutorial is to provide a detailed understanding of how interference fringes are detected and removed from `.ibw` files using PCA, while also showing how to reproduce the process locally or via the web app.

## 1️⃣ Introduction and Motivation
- Cold atom images often contain interference fringes caused by imperfections in the imaging system or environmental noise.
- These fringes reduce the signal-to-noise ratio and complicate quantitative analysis of atom distributions.
- Fringe removal is therefore critical for accurate measurement of physical parameters in cold atom experiments.
- This application uses **Principal Component Analysis (PCA)** to detect and subtract fringe patterns while preserving the atomic density information.

## 2️⃣ Mathematical Background: Principal Component Analysis (PCA)

PCA is a dimensionality reduction technique that identifies the directions (principal components) along which the variance of the data is maximized. It is widely used for noise reduction and pattern extraction.

### PCA Formulation
Given a data matrix $X$ with $n$ samples (rows) and $p$ features (columns), PCA finds orthogonal vectors $\mathbf{v}_i$ such that:

$$ \mathbf{v}_1 = \arg\max_{||v||=1} \text{Var}(Xv) $$
$$ \mathbf{v}_2 = \arg\max_{||v||=1, v \perp v_1} \text{Var}(Xv) $$
and so on, for $i = 1, ..., p$.

The principal components are the eigenvectors of the covariance matrix $\Sigma = \frac{1}{n-1} X^T X$, and the corresponding eigenvalues indicate the variance captured by each component.

### Applying PCA to Fringe Removal
- Each column of the atom image is considered a vector.
- The corresponding reference image column represents the background pattern.
- PCA identifies the dominant patterns in the reference image (typically the fringes).
- These patterns are projected onto the atom image and subtracted, effectively removing the fringe component while keeping the underlying atomic distribution.
- Columns with too few pixels are skipped to avoid computational errors (PCA requires more than 1 sample).

## 3️⃣ Accessing the Web App
The Cold Atom Fringe Remover is available as a Streamlit web application for immediate use without local installation:

[Streamlit App Link](https://cold-atom-fringe-remover-7eznbcc3gexhav8qcpsfvm.streamlit.app/)

### Web App Workflow
- Upload `.ibw` files (single or multiple files).
- The app processes each file, showing **Before** and **After** images.
- The cleaned images (`.png`) and arrays (`.npy`) are bundled in a zip for download.
- Files with unsupported structures may skip PCA for some columns.

## 4️⃣ GitHub Repository
Clone the repository to run the app locally or inspect the source code:

```bash
git clone https://github.com/ho3nik/cold-atom-fringe-remover.git
cd cold-atom-fringe-remover
```

## 5️⃣ Local Installation
1. Create a virtual environment (optional but recommended):
```bash
python -m venv venv
source venv/bin/activate  # Linux / macOS
venv\Scripts\activate    # Windows
```
2. Install dependencies:
```bash
pip install -r requirements.txt
```
3. Launch the app locally:
```bash
streamlit run streamlit_app.py
```

## 6️⃣ Uploading and Supported Files
- Upload `.ibw` files containing paired atom and reference images.
- Supported file structures:
  - 3D arrays where one axis separates the atom and reference images.
  - 2D paired images stacked side-by-side or top-bottom.
- Files with only a single image or non-standard structure may not fully process, and PCA may be skipped for certain columns.

## 7️⃣ Processing Steps Explained
**Step 1: Load `.ibw` file**
- Reads the binary wave file using `igor2`.
- Extracts the wave data containing the atom and reference images.

**Step 2: Enhance Atom Image**
- Normalizes intensity values and clips extreme values (2nd–98th percentile).
- Applies Gaussian smoothing to reduce high-frequency noise.
- Uses adaptive histogram equalization (`skimage.exposure.equalize_adapthist`) for better contrast.

**Step 3: Remove Fringes with PCA**
- Treats each column of the atom image as a sample vector.
- Computes PCA on the corresponding reference column to identify fringe patterns.
- Projects the dominant components onto the atom column and subtracts them.
- Skips PCA if the column is too short (n_samples < n_components).

**Step 4: Generate Output**
- Displays **Before** and **After** images using Matplotlib.
- Saves cleaned images (`.png`) and arrays (`.npy`).
- Bundles results into a zip file for download.

## 8️⃣ PCA Implementation in Code
Here is the Python implementation used in the fringe remover:
```python
from sklearn.decomposition import PCA
import numpy as np

def clean_fringes(atom_image, reference_image):
    # Normalize reference
    norm_ref = (reference_image - reference_image.min()) / (reference_image.ptp() + 1e-12)
    corrected = atom_image.astype(float)
    
    for i in range(atom_image.shape[1]):
        col = norm_ref[:, i].reshape(-1, 1)
        if col.shape[0] < 2:
            continue  # Skip too small columns
        pca = PCA(n_components=min(2, col.shape[0]))
        transformed = pca.fit_transform(col)
        reconstructed = pca.inverse_transform(transformed)
        corrected[:, i] -= reconstructed.flatten()
    
    # Normalize corrected image
    return (corrected - corrected.min()) / (corrected.ptp() + 1e-12)
```

**Explanation:**
- `PCA.fit_transform()` identifies the dominant pattern (fringes) in the reference column.
- `inverse_transform()` reconstructs the pattern in the original feature space.
- Subtracting the reconstruction removes fringes from the atom image.
- Normalization ensures pixel values remain between 0 and 1.

## 9️⃣ Results and Limitations
- **Before:** Original atom image with interference fringes.
- **After:** Enhanced and cleaned image with fringes removed.
- Outputs include `.png` visualizations and `.npy` numerical arrays.
- PCA may be skipped for very small columns (n_samples < 2) or files with unconventional structure.
- Very large images may require longer processing time; consider downsampling if needed.

## 🔟 Optional: Programmatic Use
Process `.ibw` files using the Python API:
```python
from igor2.binarywave import load as load_ibw
from fringe_remover import enhance, clean_fringes

data = load_ibw('PF_01Apr2024_0588.ibw')
w = data['wave']['wData']
ipwa = w[:,:,0].T
ipwoa = w[:,:,1].T

before = enhance(ipwa)
after = clean_fringes(ipwa, ipwoa)
```
- This snippet reproduces the exact cleaning process programmatically.
- Useful for batch processing or further analysis in custom scripts.

##Author
**H. RJ Nikzat**